<a href="https://colab.research.google.com/github/lharikumar/langchain_intro/blob/main/LangChainModules.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Do the required installs 

In [ ]:
!pip install -qU openai python-dotenv langchain tiktoken faiss-cpu datasets wikipedia google-search-results

### Load the key

In [ ]:
import os
import dotenv

In [ ]:
dotenv.load_dotenv('/content/variables/.env')
openai_api_key = os.getenv('OPENAI_API_KEY')

In [ ]:
#openai_api_key = "Your_api_key"

# Models

### The different types of AI models that are supported in LangChain

*   **LLMs** - These models take a text string as input, and return a text string as output.
*   **Chat Models** - These models take a list of Chat Messages as input, and return a Chat Message.
*   **Text Embeddings** - These models take text as input and return a list of floats.



## LLMs

In [ ]:
import langchain

In [ ]:
from langchain.llms import OpenAI
llm = OpenAI(model_name="gpt-3.5-turbo", temperature=0.2)
llm("Tell me a story about animals in 50 words or less")

## Chat Models

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    SystemMessage,                       # Provide context to the AI 
    HumanMessage,                        # User query
    AIMessage                            # Response from AI Model
)

In [ ]:
chat = ChatOpenAI(temperature=0)

chat(
       [
        SystemMessage(content="You're an AI bot that can recommend Italian food"),
        HumanMessage(content="I am going to Italy next week and want to try out some Italian food. What should I eat")
       ] 
    )


## Text Embeddings

In [ ]:
## Change the text into a embedding (sequence of numbers that hold the semantic meaning).
from langchain.embeddings.openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

# Note that there are different embedding methods for documents (to be searched over) vs queries (the search query itself).
query = "This is a sample query"
query_embedding = embeddings.embed_query(query) ## works over single document


In [ ]:
len(query_embedding)

In [ ]:
query_embedding[0]

# Prompts


*   **LLM Prompt Templates** - Reproducible way to generate a prompt. It contains a text string (“the template”), that can take in a set of parameters from the end user and generate a prompt.
*   **Chat Templates** - Chat Models takes a list of chat messages as input - this list commonly referred to as a prompt. These chat messages differ from raw string (which you would pass into a LLM model) in that every message is associated with a role.
*   **Example Selectors** - If you have a large number of examples, you may need to select which ones to include in the prompt. The ExampleSelector is the class responsible for doing so.
* **Output Parsers** - Language models output text. But many times you may want to get more structured information than just text back. This is where output parsers come in.




## Prompt Templates

In [ ]:
from langchain import PromptTemplate
from langchain.llms import OpenAI
llm = OpenAI(model_name="gpt-3.5-turbo", temperature=0.0)

template = """
I am travelling to France next week. What are the best museums to visit at {city} ?

Respond in 50 words or less.
"""

prompt = PromptTemplate(
    input_variables=['city'],
    template=template,
    validate_template=True,
)
prompt.format(city='Paris')

print(f"Prompt for the  llm: {prompt.format(city='Paris')}")

In [ ]:
llm(prompt.format(city='Paris'))

## Chat Prompt Templates

In [ ]:
from langchain.prompts import SystemMessagePromptTemplate, AIMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate

# from langchain.schema import AIMessage, HumanMessage,  SystemMessage

template="You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt=SystemMessagePromptTemplate.from_template(template)

human_template="{text}"
human_message_prompt=HumanMessagePromptTemplate.from_template(human_template)

ai_template="I love Programming : J'adore programmer"
ai_message_prompt=AIMessagePromptTemplate.from_template(ai_template)

chat_prompt=ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt, ai_message_prompt])
chat_prompt_formatted=chat_prompt.format_prompt(input_language="English", output_language="French", text="I love to eat Pizzas").to_messages()

In [ ]:
from langchain.chat_models import ChatOpenAI
chat = ChatOpenAI(temperature=0)
chat(chat_prompt_formatted)

## Example selectors

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.prompts import FewShotPromptTemplate

In [ ]:
## LengthBased Example Selector -This ExampleSelector selects which examples to use based on length. 
from langchain.prompts.example_selector import LengthBasedExampleSelector

In [ ]:
# These are a lot of examples 
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "energetic", "output": "lethargic"},
    {"input": "sharp", "output": "blunt"},
    {"input": "windy", "output": "calm"},
]

In [ ]:
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)
example_selector = LengthBasedExampleSelector(    
    examples=examples,              # Available examples
    example_prompt=example_prompt,  # Prompt Template
    max_length=25,                  # Length of the string
)
dynamic_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Give the antonym of every input",
    suffix="Input: {input_word}\nOutput:", 
    input_variables=["input_word"],
)

In [ ]:
# An example with small input, so it selects all examples.
print(dynamic_prompt.format(input_word="big"))

In [ ]:
# An example with long input, so it selects only one example.
long_string = "big and huge and massive and large and gigantic and tall and much much much much much bigger than everything else"
print(dynamic_prompt.format(input_word=long_string))

## Semantic Similarity Example Selector


In [ ]:
## The **SemanticSimilarityExampleSelector** selects examples based on which examples are most similar to the inputs. 
## It does this by finding the examples with the embeddings that have the greatest cosine similarity with the inputs.
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import FAISS

In [ ]:
# These are a lot of examples of a task of creating antonyms.
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "energetic", "output": "lethargic"},
    {"input": "sharp", "output": "blunt"},
    {"input": "windy", "output": "calm"},
]
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)

In [ ]:
embeddings=OpenAIEmbeddings()

example_selector = SemanticSimilarityExampleSelector.from_examples(   
    examples=examples,
    embeddings=embeddings,
    vectorstore_cls=FAISS,
    k=2
)
dynamic_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Give the antonym of every input",
    suffix="Input: {input_word}\nOutput:", 
    input_variables=["input_word"],
)

In [ ]:
print(dynamic_prompt.format(input_word="enthusiastic"))


### Other Example Selectors

*   The **MaxMarginalRelevanceExampleSelector** selects examples based on a combination of which examples are most similar to the inputs, while also optimizing for diversity. It does this by finding the examples with the embeddings that have the greatest cosine similarity with the inputs, and then iteratively adding them while penalizing them for closeness to already selected examples.
*   The **NGramOverlapExampleSelector** selects and orders examples based on which examples are most similar to the input, according to an ngram overlap score. The ngram overlap score is a float between 0.0 and 1.0, inclusive.


In [ ]:
### Custom Example Selector

In [ ]:
from langchain.prompts.example_selector.base import BaseExampleSelector
from typing import Dict, List
import numpy as np


class CustomExampleSelector(BaseExampleSelector):    
    def __init__(self, examples: List[Dict[str, str]]):
        self.examples = examples
    
    def add_example(self, example: Dict[str, str]) -> None:
        """Add new example to store for a key."""
        self.examples.append(example)

    def select_examples(self, input_variables: Dict[str, str]) -> List[dict]:
        """Select which examples to use based on the inputs."""
        return np.random.choice(self.examples, size=2, replace=False)


In [ ]:
examples = [
    {"foo": "1"},
    {"foo": "2"},
    {"foo": "3"},
    {"foo": "11"},
    {"foo": "12"},
    {"foo": "13"},    
    {"foo": "111"},
    {"foo": "112"},
    {"foo": "113"}
]

# Initialize example selector.
example_selector = CustomExampleSelector(examples)

# Select examples
selected_values=example_selector.select_examples({"foo": "foo"})
print(f"First selection: {selected_values}")

# Add new example to the set of examples
example_selector.add_example({"foo": "44444"})
print(f"Updated examples: {example_selector.examples}")

# # Select examples
# example_selector.select_examples({"foo": "foo"})
print(f"Second selection: {selected_values}")

## Output Parsers

Language models output text. But many times you may want to get more structured information than just text back. This is where output parsers come in.

*   CommaSeparatedListOutputParser - output will be a comma separated list
*   OutputFixingParser - Parse the LLM response. In case of errors, we can pass the misformatted output, along with the formatted instructions, to the model and ask it to fix it
*   PydanticOutputParser - allows users to specify an arbitrary JSON schema and query LLMs for JSON outputs that conform to that schema.
*   RetryOutputParser - when the output is not just in the incorrect format, but is partially complete,  the RetryOutputParser, which passes in the prompt (as well as the original output) to try again to get a better response.
*   Structured Output Parser - Define a response schema and convert them to instructions to pass to the prompt

In [ ]:
## Structed Output Parser
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

In [ ]:
response_schemas = [
    ResponseSchema(name="answer", description="answer to the user's question"),
    ResponseSchema(name="source", description="source used to answer the user's question, should be a website.")
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
output_parser.get_format_instructions()

In [ ]:
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="answer the users question as best as possible.\n{format_instructions}\n{question}",
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions}
)

In [ ]:
llm = OpenAI(temperature=0)
_input = prompt.format_prompt(question="what's the capital of france")
output = llm(_input.to_string())
output_parser.parse(output)

# Memory
In chats, it is highly important to remember previous interactions, both at a short term but also at a long term level. The concept of “Memory” exists to do exactly that.
Memory involves keeping a concept of state around throughout a user’s interactions with an language model

Types of memory - https://python.langchain.com/en/latest/modules/memory/how_to_guides.html

# Indexes 

Indexes refer to ways to structure documents so that LLMs can best interact with them. 

### Document Loaders
Import data from other sources. For the complete list of loaders refer to https://python.langchain.com/en/latest/modules/indexes/document_loaders.html

In [ ]:
from langchain.document_loaders import HuggingFaceDatasetLoader
dataset_name="imdb"
page_content_column="text"

loader=HuggingFaceDatasetLoader(dataset_name,page_content_column)
data = loader.load()

In [ ]:
data[10]

## Text Splitters
When you want to deal with long pieces of text, it is necessary to split up that text into chunks. For the complete list of text splitters supported refer to https://python.langchain.com/en/latest/modules/indexes/text_splitters.html

### RecursiveCharacterTextSplitter
This text splitter is the recommended one for generic text. It is parameterized by a list of characters. It tries to split on them in order until the chunks are small enough. The default list is ["\n\n", "\n", " ", ""]. This has the effect of trying to keep all paragraphs (and then sentences, and then words) together as long as possible, as those would generically seem to be the strongest semantically related pieces of text.

How the text is split: by list of characters

How the chunk size is measured: by length function passed in (defaults to number of characters)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

with open('/content/data/samplefile.txt') as f:
  file_data = f.read();

print(f"File Contents first 100 characters: {file_data[:100]}")

text_splitter = RecursiveCharacterTextSplitter(chunk_size=25,
                                               chunk_overlap=5)
split_text=text_splitter.create_documents([file_data])
print(f"First chunk:  {split_text[0]}")
print(f"Second chunk: {split_text[1]}")


## VectorStores
The most common type of index is one that creates numerical embeddings (with an Embedding Model) for each document. A vectorstore stores Documents and associated embeddings, and provides fast ways to look up relevant Documents by embeddings.

## Retrievers
The retriever interface is a generic interface that makes it easy to combine documents with language models. This interface exposes a get_relevant_documents method which takes in a query (a string) and returns a list of documents.
Complete list of retrievers is available at https://python.langchain.com/en/latest/modules/indexes/retrievers.html

### Wikipedia Retrievers 
To retrieve wiki pages from wikipedia.org into the Document format that is used downstream.

In [ ]:
from langchain.retrievers import WikipediaRetriever
retriever = WikipediaRetriever()
docs=retriever.get_relevant_documents(query=("Midjourney"))
docs[0]

## Chains
 Chains allow us to combine multiple components together to create a single, coherent application.
 A chain is just an end-to-end wrapper around multiple individual components.


*   LLM Chain - A LLMChain is the most common type of chain. It consists of a PromptTemplate, a model (either an LLM or a ChatModel), and an optional output parser. This chain takes multiple input variables, uses the PromptTemplate to format them into a prompt. It then passes that to the model. Finally, it uses the OutputParser (if provided) to parse the output of the LLM into a final format.
*   Index related chain - This category of chains are used for interacting with indexes. The purpose these chains is to combine your own data (stored in the indexes) with LLMs. The best example of this is question answering over your own documents.

For the complete list of chains refer to https://python.langchain.com/en/latest/modules/chains/how_to_guides.html

In [ ]:
## Loading from LangChainHub
from langchain.chains import load_chain

chain = load_chain("lc://chains/llm-math/chain.json")
chain.run("whats 2 raised to .12")

## Agents

Some applications will require not just a predetermined chain of calls to LLMs/other tools, but potentially an unknown chain that depends on the user’s input. In these types of chains, there is a “agent” which has access to a suite of tools. Depending on the user input, the agent can then decide which, if any, of these tools to call.

At the moment, there are two main types of agents:

“Action Agents”: these agents decide an action to take and take that action one step at a time

“Plan-and-Execute Agents”: these agents first decide a plan of actions to take, and then execute those actions one at a time.

*   **Agents** use an LLM to determine which actions to take and in what order. An action can either be using a tool and observing its output, or returning a response to the user.
*  **Tools:** these are the actions an agent can take. What tools you give an agent highly depend on what you want the agent to do
*  **Toolkits** an agent applied to a particular use case.

https://python.langchain.com/en/latest/modules/agents/agents/agent_types.html - List of agents available


In [ ]:
## Plan and execute agents accomplish an objective by first planning what to do, then executing the sub tasks.

from langchain.chat_models import ChatOpenAI
from langchain.experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner
from langchain.llms import OpenAI
from langchain import SerpAPIWrapper
from langchain.agents.tools import Tool
from langchain import LLMMathChain

serpapi_api_key=os.getenv("SERPAPI_API_KEY")
search = SerpAPIWrapper()
llm = OpenAI(temperature=0)
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math"
    ),
]

model = ChatOpenAI(temperature=0)
planner = load_chat_planner(model)
executor = load_agent_executor(model, tools, verbose=True)
agent = PlanAndExecute(planner=planner, executer=executor, verbose=True)

agent.run("Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?")

In [ ]:
## Conversation Agent

from langchain.agents import Tool
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.utilities import SerpAPIWrapper
from langchain.agents import initialize_agent
from langchain.agents import AgentType

search = SerpAPIWrapper()
tools = [
    Tool(
        name = "Current Search",
        func=search.run,
        description="useful for when you need to answer questions about current events or the current state of the world. the input to this should be a single search term."
    ),
]
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
llm=ChatOpenAI(temperature=0)
agent_chain = initialize_agent(tools, llm, agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION, verbose=True, memory=memory)
agent_chain.run(input="Hi, i am Jane")

In [ ]:
agent_chain.run(input="who am i?")

## Callbacks
LangChain provides a callbacks system that allows you to hook into the various stages of your LLM application. This is useful for logging, monitoring, streaming, and other tasks.

In [ ]:
## StdoutHandler
## StdOutCallbackHandler, which simply logs all events to stdout.
## when the verbose flag on the object is set to true, the StdOutCallbackHandler will be invoked even without being explicitly passed in.

from langchain.callbacks import StdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

handler = StdOutCallbackHandler()
llm = OpenAI()
prompt = PromptTemplate.from_template("1 + {number} = ")

# First, let's explicitly set the StdOutCallbackHandler in `callbacks`
chain = LLMChain(llm=llm, prompt=prompt, callbacks=[handler])
chain.run(number=2)

# Then, let's use the `verbose` flag to achieve the same result
chain = LLMChain(llm=llm, prompt=prompt, verbose=True)
chain.run(number=2)

# Finally, let's use the request `callbacks` to achieve the same result
chain = LLMChain(llm=llm, prompt=prompt)
chain.run(number=3, callbacks=[handler])